<a href="https://colab.research.google.com/github/snest21/TP-Model/blob/master/2020_1_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%js
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click()
}setInterval(ClickConnect,60000)

In [0]:
a = []
while True:
    a.append(1)

In [1]:
!git clone https://github.com/ccasimiro88/TranslateAlignRetrieve.git

Cloning into 'TranslateAlignRetrieve'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 351 (delta 210), reused 236 (delta 112), pack-reused 0
Receiving objects: 100% (351/351), 14.50 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (210/210), done.


In [0]:
%%capture
!pip install cdqa

## Running the model

In [0]:
import os
import torch
import joblib
from cdqa.reader import BertProcessor, BertQA

In [0]:
%%time
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='./TranslateAlignRetrieve/SQuAD-es-v1.1/train-v1.1-es.json')

100%|██████████| 231508/231508 [00:00<00:00, 1226491.98B/s]


CPU times: user 16min 18s, sys: 4.01 s, total: 16min 22s
Wall time: 16min 23s


In [0]:
%%time
reader = BertQA(train_batch_size=12,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir='models')

reader.fit(X=(train_examples, train_features))

100%|██████████| 440473133/440473133 [00:12<00:00, 36324989.69B/s]


## Save the model

In [0]:
reader.model.to('cpu')
reader.device = torch.device('cpu')
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa2.joblib'))

In [0]:
!cp -a ./models ./drive/My\ Drive/

## Get the model

In [0]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp ./drive/My\ Drive/bert_qa.joblib ./

## Load the model

In [0]:
from cdqa.pipeline import QAPipeline

In [0]:
cdqa_pipeline = QAPipeline(reader='./bert_qa.joblib')

100%|██████████| 231508/231508 [00:00<00:00, 898704.28B/s]


## Testing the model

In [0]:
%%time
from cdqa.utils.evaluation import evaluate_reader
from cdqa.utils.download import *

cdqa_pipeline = QAPipeline(reader='./models/bert_qa2.joblib')
eval_dict = evaluate_reader(cdqa_pipeline, "./TranslateAlignRetrieve/SQuAD-es-v1.1/dev-v1.1-es_small.json")

print(eval_dict)

{'exact_match': 37.88546255506608, 'f1': 54.0622231996906}
CPU times: user 2min 33s, sys: 41.1 s, total: 3min 14s
Wall time: 3min 14s


In [0]:
!head -n 30 ./models/predictions.json

{
    "56be4db0acb8001400a502ec": "campe\u00f3n de la American Football Conference",
    "56be4db0acb8001400a502ee": "campe\u00f3n de la NFL para la temporada 2015",
    "56be4db0acb8001400a502ef": "temporada 2015",
    "56be4db0acb8001400a502f0": "oro",
    "56be8e613aeaaa14008c90d2": "7 de febrero de 2016",
    "56bea9923aeaaa14008c91b9": "aniversario dorado",
    "56bea9923aeaaa14008c91bb": "7 de febrero de 2016",
    "56beace93aeaaa14008c91df": "Carolina Panthers 24-10",
    "56beace93aeaaa14008c91e2": "juego",
    "56beace93aeaaa14008c91e3": "temporada 2015",
    "56bf10f43aeaaa14008c94fd": "NFC",
    "56bf10f43aeaaa14008c9501": "oro",
    "56d20362e7d4791d009025e8": "2015",
    "56d20362e7d4791d009025e9": "Denver Broncos derrot\u00f3 al campe\u00f3n de la National Football Conference",
    "56d20362e7d4791d009025eb": "Carolina Panthers 24-10",
    "56d600e31c85041400946eae": "temporada 2015",
    "56d600e31c85041400946eb0": "Carolina Panthers 24-10 para ganar su tercer t\u00edtul

In [0]:
!head -c 20060 ./TranslateAlignRetrieve/SQuAD-es-v1.1/dev-v1.1-es_small.json

{"version": "1.1", "data": [{"title": "Super Bowl _ 50", "paragraphs": [{"context": "El Super Bowl 50 fue un partido de f\u00fatbol americano para determinar al campe\u00f3n de la NFL para la temporada 2015. El campe\u00f3n de la American Football Conference (AFC) Denver Broncos derrot\u00f3 al campe\u00f3n de la National Football Conference (NFC) Carolina Panthers 24-10 para ganar su tercer t\u00edtulo de Super Bowl. El partido se jug\u00f3 el 7 de febrero de 2016 en el Levi 's Stadium en la Bah\u00eda de San Francisco en Santa Clara, California. Como este fue el 50\u00ba Super Bowl, la liga enfatiz\u00f3 el \"aniversario dorado\" con varias iniciativas tem\u00e1ticas de oro, as\u00ed como suspender temporalmente la tradici\u00f3n de nombrar cada juego de Super Bowl con n\u00fameros romanos (bajo los cuales el juego).", "qas": [{"question": "\u00bfQu\u00e9 equipo de la NFL represent\u00f3 a la AFC en el Super Bowl 50?", "answers": [{"text": "Denver Broncos", "answer_start": 165}, {"te

In [0]:
!bash ./TranslateAlignRetrieve/src/qa/setup_env.sh

In [0]:
!pip install transformers

In [0]:
!bash ./TranslateAlignRetrieve/src/qa/train_m-bert.sh ./TranslateAlignRetrieve/SQuAD-es-v1.1/train-v1.1-es.json v1


Iteration:  13% 1458/11246 [08:39<57:53,  2.82it/s]
Iteration:  13% 1459/11246 [08:39<57:56,  2.82it/s]
Iteration:  13% 1460/11246 [08:40<57:54,  2.82it/s]
Iteration:  13% 1461/11246 [08:40<57:53,  2.82it/s]
Iteration:  13% 1462/11246 [08:40<57:54,  2.82it/s]
Iteration:  13% 1463/11246 [08:41<57:52,  2.82it/s]
Iteration:  13% 1464/11246 [08:41<57:50,  2.82it/s]
Iteration:  13% 1465/11246 [08:41<57:51,  2.82it/s]
Iteration:  13% 1466/11246 [08:42<57:49,  2.82it/s]
Iteration:  13% 1467/11246 [08:42<57:47,  2.82it/s]
Iteration:  13% 1468/11246 [08:42<57:52,  2.82it/s]
Iteration:  13% 1469/11246 [08:43<57:53,  2.81it/s]
Iteration:  13% 1470/11246 [08:43<57:51,  2.82it/s]
Iteration:  13% 1471/11246 [08:43<57:54,  2.81it/s]
Iteration:  13% 1472/11246 [08:44<57:51,  2.82it/s]
Iteration:  13% 1473/11246 [08:44<57:49,  2.82it/s]
Iteration:  13% 1474/11246 [08:45<57:55,  2.81it/s]
Iteration:  13% 1475/11246 [08:45<57:53,  2.81it/s]
Iteration:  13% 1476/11246 [08:45<57:49,  2.82it/s]
Iteration: 

In [0]:
!bash ./TranslateAlignRetrieve/src/qa/download_mlqa_xquad.sh

--2020-04-08 19:21:51--  https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75719050 (72M) [application/zip]
Saving to: ‘/content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1.zip’

MLQA_V1.zip         100%[===================>]  72.21M  22.6MB/s    in 3.5s    

2020-04-08 19:21:55 (20.6 MB/s) - ‘/content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1.zip’ saved [75719050/75719050]

Archive:  /content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1.zip
   creating: /content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1/
   creating: /content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1/dev/
  inflating: /content/TranslateAlignRetrieve/src/qa/corpora/MLQA_V1/dev/dev-context-ar-question-ar.json  
  inflating: /content/Translate

In [0]:
!bash ./TranslateAlignRetrieve/src/qa/evaluate_m-bert.sh ./TranslateAlignRetrieve/src/qa/data/training/m-bert_train-v1.1-es_small.json/ es es mlqa

bash: ./TranslateAlignRetrieve/src/qa/evaluate_m-bert.sh: No such file or directory
